In [ ]:
!wget https://lfs.aminer.cn/lab-datasets/citation/DBLP-citation-network-Oct-19.tar.gz
!tar -zxvf ./DBLP-citation-network-Oct-19.tar.gz

In [3]:
import glob
import json
import csv

articles = {}
authors = set()
venues = set()


def write_header(file, fields):
    csv.writer(file, delimiter=",").writerow(fields)


with open("data/article_REFERENCES_article.csv", "w") as article_references_article_file, \
     open("data/article_REFERENCES_article_header.csv", "w") as article_references_article_header_file, \
     open("data/article_AUTHOR_author.csv", "w") as article_author_author_file, \
     open("data/article_AUTHOR_author_header.csv", "w") as article_author_author_header_file, \
     open("data/article_VENUE_venue.csv", "w") as article_venue_venue_file, \
     open("data/article_VENUE_venue_header.csv","w") as article_venue_venue_header_file:

    write_header(article_references_article_header_file, [":START_ID(Article)", ":END_ID(Article)"])
    write_header(article_author_author_header_file, [":START_ID(Article)", ":END_ID(Author)"])
    write_header(article_venue_venue_header_file, [":START_ID(Article)", ":END_ID(Venue)"])

    articles_references_article_writer = csv.writer(article_references_article_file, delimiter=",")
    article_author_author_file_writer = csv.writer(article_author_author_file, delimiter=",")
    article_venue_venue_file_writer = csv.writer(article_venue_venue_file, delimiter=",")

    for file_path in glob.glob("dblp-ref/*.json"):
        print(file_path)
        with open(file_path, "r") as file:
            line = file.readline()
            while line:
                item = json.loads(line)
                articles[item["id"]] = {"abstract": item.get("abstract", ""),
                                        "title": item["title"], "year": item["year"]}

                venue = item["venue"]
                if venue:
                    venues.add(venue)
                    article_venue_venue_file_writer.writerow([item["id"], venue])

                for reference in item.get("references", []):
                    articles_references_article_writer.writerow([item["id"], reference])

                for author in item.get("authors", []):
                    authors.add(author)
                    article_author_author_file_writer.writerow([item["id"], author])

                line = file.readline()

with open("data/articles.csv", "w") as articles_file, \
     open("data/articles_header.csv", "w") as articles_header_file, \
     open("data/authors.csv", "w") as authors_file, \
     open("data/authors_header.csv", "w") as authors_header_file, \
     open("data/venues.csv", "w") as  venues_file, \
     open("data/venues_header.csv","w") as venues_header_file:
    write_header(articles_header_file, ["index:ID(Article)", "title:string", "abstract:string", "year:int"])
    write_header(authors_header_file, ["name:ID(Author)",])
    write_header(venues_header_file, ["name:ID(Venue)"])

    articles_writer = csv.writer(articles_file, delimiter=",")
    for article_id in articles:
        article = articles[article_id]
        articles_writer.writerow([article_id, article["title"], article["abstract"], article.get("year")])

    authors_writer = csv.writer(authors_file, delimiter=",")
    for author in authors:
        authors_writer.writerow([author])

    venues_writer = csv.writer(venues_file, delimiter=",")
    for venue in venues:
        venues_writer.writerow([venue])


dblp-ref/dblp-ref-2.json
dblp-ref/dblp-ref-3.json
dblp-ref/dblp-ref-0.json
dblp-ref/dblp-ref-1.json


In [ ]:
s= """./bin/neo4j-admin import \
  --database=citations.db \
  --nodes:Author=${DATA_DIR}/authors_header.csv,${DATA_DIR}/authors.csv \
  --nodes:Article=${DATA_DIR}/articles_header.csv,${DATA_DIR}/articles.csv \
  --nodes:Venue=${DATA_DIR}/venues_header.csv,${DATA_DIR}/venues.csv \
  --relationships:REFERENCES=${DATA_DIR}/article_REFERENCES_article_header.csv,${DATA_DIR}/article_REFERENCES_article.csv \
  --relationships:AUTHOR=${DATA_DIR}/article_AUTHOR_author_header.csv,${DATA_DIR}/article_AUTHOR_author.csv \
  --relationships:VENUE=${DATA_DIR}/article_VENUE_venue_header.csv,${DATA_DIR}/article_VENUE_venue.csv \
  --multiline-fields=true"""

In [ ]:
s.format(DATA_DIR="/home/kent/opt/citations").replace("$","").replace(":","=")

In [10]:
s.format(DATA_DIR="/home/kent/opt/citations").replace("$","").replace(":","=")

'./bin/neo4j-admin import   --database=citations.db   --nodes=Author=/home/kent/opt/citations/authors_header.csv,/home/kent/opt/citations/authors.csv   --nodes=Article=/home/kent/opt/citations/articles_header.csv,/home/kent/opt/citations/articles.csv   --nodes=Venue=/home/kent/opt/citations/venues_header.csv,/home/kent/opt/citations/venues.csv   --relationships=REFERENCES=/home/kent/opt/citations/article_REFERENCES_article_header.csv,/home/kent/opt/citations/article_REFERENCES_article.csv   --relationships=AUTHOR=/home/kent/opt/citations/article_AUTHOR_author_header.csv,/home/kent/opt/citations/article_AUTHOR_author.csv   --relationships=VENUE=/home/kent/opt/citations/article_VENUE_venue_header.csv,/home/kent/opt/citations/article_VENUE_venue.csv   --multiline-fields=true'